<a href="https://colab.research.google.com/github/anudaweerasinghe/evo-genome-classification/blob/add-naive-training/distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (41

In [15]:
!pip install numpy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 39.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
from datasets import load_dataset
genomes = load_dataset("anudaw/genome-classification")

In [13]:
id2label = {0: "H", 1: "I", 2: "C"}
label2id = {"H" : 0, "I": 1, "C": 2}


In [14]:
def update_label_to_int(examples):
    return {"seq" : examples["seq"], 'label': [label2id[label] for label in examples['label']]}

genomes = genomes.map(update_label_to_int, batched=True)
print(genomes['train'][0])

{'seq': 'ATCTAGAATATAGAAAAGTGGTAAGACAAAAAAGAATAAACTGGTTATTTGATAGAATAAGAGAAAGAGCAGAAGACAGTGGCAATGAGAGCGAGGGAGATACAGAGGAATTGGCAGCACTTGGGGAAACGGGGCATCTTATTCTTGGGGATATTAATAATCTGTAGCGCTGCAAACAACTTGTGGGTCACAGTCTATTATGGGGTACCTGTGTGGAAAGAAGCAACAACCACTCTATTCTGTGCATCAGATGCTAAAGCATATGAAACAGAGGTACATAATGTCTGGGCTACACATGCCTGTGTACCCACGGACCCCAACCCACAAGAAGTAGTTTTGGAAAATGTAACAGAAAATTTTAACATGTGGAAAAATAACATGGTAGAGCAGATGCATACAGATATAATCAGTTTATGGGATGAAAGCCTAAAACCATGTGTAAAGTTAACCCCTCTCTGTGTCACTCTAACTTGTACCAATGTAACTAATAACCGTACCAATGCAAATAAGAACGATACCAATATCAATGCAACTGTGACATCAACTGACGAGATAAAAAATTGCTCTTTCAATATAACCACAGAACTAAAAGATAAGAAAAAGAGAGTGTCTGCACTTTTCTATAAGCTTGATATAGTGCAAATAAAGCAATCTGAAATTAATCAATCTGAATCTGAAGATAGGCTAATAAATTGTAATACCTCAACCGTTACACAGGCCTGTCCAAAGGTATCCTTTGAGCCAATTCCCATACATTATTGTGCTCCAGCTGGCTTTGCGATTCTAAAGTGTAATAATAACACATGCAATGGAACAGGGCCATGCACAAATGTCAGCACAGTACAATGTACACATGGAATTAAACCAGTGGTATCAACTCAATTGCTATTAAATGGCAGCCTAGCAGAAGAAGAGATAATTATTAGGTCTGAAGATATTACAAAGAATACAAAAAACATAATAGTACAGCTTAATGAAGCTGTAGAAATTA

In [25]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/canine-c")

tokenizer_config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

In [26]:
def preprocess_function(examples):
    # print(len(examples['seq']))
    return tokenizer(examples["seq"], truncation=True)

tokenized_genomes = genomes.map(preprocess_function, batched=True)

Map:   0%|          | 0/48000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [27]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [28]:
import evaluate
accuracy = evaluate.load("accuracy")

In [29]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return accuracy.compute(predictions=predictions, references=labels)



In [31]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "google/canine-c", num_labels=3, id2label=id2label, label2id=label2id
)



config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/529M [00:00<?, ?B/s]

Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-c and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
training_args = TrainingArguments(
    output_dir = "model_data",
    learning_rate=2e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_genomes["train"],
    eval_dataset=tokenized_genomes["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.101800,1.102531,0.333333


Checkpoint destination directory model_data/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
